### 1. Importación de librerias necesarias para el proyecto

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
#import numpy as np
#import cv2
from PIL import Image
import requests
import matplotlib.pyplot as plt

ImportError: cannot import name 'broadcast_to' from 'numpy.lib.stride_tricks' (c:\Users\inter\miniconda3\envs\redes_neuronales\lib\site-packages\numpy\lib\stride_tricks.py)